# Notebook to process the Michigan Claims Loss notes
### I used Cause of Loss (COL) = 400 and dates after 2009-12-31


### SQL code to generate this file
#### Step 1
```
drop table if exists kesj.claims_400;
create table kesj.claims_400 as
        select distinct
                systems_fdwatomcecs.col.rsrv_col_cd,
                systems_fdwatomcecs.col.clm_id
         from
                systems_fdwatomcecs.col
         where
                systems_fdwatomcecs.col.rsrv_col_cd ='400';
```
#### Step 2
```
drop table if exists kesj.claims_400_more;
create table kesj.claims_400_more as
select distinct 
        kesj.claims_400.rsrv_col_cd,
        kesj.claims_400.clm_id,
        systems_fdwatomcecs.clm.clm_num,
        systems_fdwatomcecs.clm.clm_st_cd,
		systems_fdwatomcecs.clm.st_asgn_st_cd,
		systems_fdwatomcecs.clm.los_occr_dt,
		systems_fdwatomcecs.clm.los_type_cd,
		systems_fdwatomcecs.clm.rpt_agt_st_cd,
		systems_fdwatomcecs.clm.los_loc_city_nm,
		systems_fdwatomcecs.clm.los_loc_stret_nm,
		systems_fdwatomcecs.clm.los_loc_st_cd,
		systems_fdwatomcecs.clm.los_loc_desc_txt,
        systems_fdwatomcecs.clm.los_desc_txt,
        systems_fdwatomcecs.clm.los_rpt_dt,
        systems_fdwatomcecs.clm.app_cd,
        systems_fdwatomcecs.clm.user_type_cd,
		systems_fdwatomcecs.clm.loc_qlty_cd,
		systems_fdwatomcecs.clm.latud_num,
		systems_fdwatomcecs.clm.lngtd_num
FROM
        kesj.claims_400
LEFT OUTER JOIN 
        systems_fdwatomcecs.clm
ON
 (        kesj.claims_400.clm_id = systems_fdwatomcecs.clm.clm_id) ;
        ```
#### step 3: 
```
drop table if exists kesj.claims_400_mi;
create table kesj.claims_400_mi as
select * 
FROM
    kesj.claims_400_more
WHERE
	kesj.claims_400_more.los_loc_st_cd = 'MI' 
AND kesj.claims_400_more.los_rpt_dt > '2009-12-31' ;

```

### generic loads

In [2]:
import os,subprocess
import pandas as pd
import numpy as np
%matplotlib inline
#from datetime import datetime
import matplotlib.pyplot as plt
#from sklearn import preprocessing
#from itertools import chain
#import random
plt.style.use('fivethirtyeight') # Good looking plots
#import seaborn as sns

### check on the paths

In [3]:
# check if the local path exists
dir1local = '/home/kesj/work/mi400test1'
if not os.path.exists(dir1local):
    # make the local directory
    !mkdir {dir1local}

%pwd
%cd {dir1local}
cwd=os.path.abspath(os.curdir)
print cwd

/home/kesj/work/mi400test1
/home/kesj/work/mi400test1


In [16]:
def hdfs_path_does_exist(path):
    return subprocess.call(['hdfs','dfs','-ls',path])
    # returns 0 if does_exist; 1 otherwise
    
## function to load into pandas from hdfs (by copying to local filespace)
def pandas_read_hdfs(infile,sep = ';',**kwargs):
    # copy the infile to the cwd
    !hdfs dfs -get {infile} .
    # identify the local file name
    inname = infile[infile.rfind('/')+1:]
    # read into a data frame
    #if dtype_dict != None:
    #    df = pd.read_csv(inname,sep=sep,**kwargs)
    #else:
    df = pd.read_csv(inname,sep=sep,**kwargs)
    # clean up local filespace
    !rm {inname}
    return df

In [9]:
# check if the remote (HDFS) path exists
hdfsdir = '/user/kesj/data/mi400'
if not hdfs_path_does_exist(hdfsdir):
    fnames = !hdfs dfs -ls {hdfsdir}
    infilenames = [f.split()[-1] for f in fnames[1:]]

print infilenames


['/user/kesj/data/mi400/mi400a.ssv']


In [17]:
mi400 = pandas_read_hdfs(infilenames[0],header=None)

get: `mi400a.ssv': File exists


In [18]:
len(mi400)

616881

In [19]:
mi400.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,400,0000052048071,004W253,22,,2010-12-28,REG,22,BASTTLE CREEK,,MI,PARKING RAMP,INSURED NOTICED DAMAGE WHEN RETURNING TO CAR,2011-01-06,I,AGT,,0,0
1,400,0000033413931,002V719,22,,2010-11-20,REG,,Bay City,,MI,probably in parking lot off EUCLID,"Final\rWhile V1 was parked and unoccupied, it ...",2010-11-20,E,CRC,,0,0
2,400,0000033413931,002V719,22,,2010-11-20,REG,,Bay City,,MI,probably in parking lot off EUCLID,"While V1 was parked and unoccupied, it was hit...",2010-11-20,E,CRC,,0,0
3,400,0000034590923,002Z262,22,,2010-11-23,REG,22,BENTON HARBOR,,MI,I-94 WEST BOUND,"V1 ON I94 NEAR EXIT RAMP. V2 STARTED TO EXIT,...",2010-11-24,I,AGT,,0,0
4,400,0000034590923,002Z262,22,,2010-11-23,REG,22,BENTON HARBOR,,MI,I-94 WEST BOUND,"V1 ON I94 NEAR EXIT RAMP. V2 STARTED TO EXIT,...",2010-11-24,I,AGT,,0,0


In [20]:
mi400.shape

(616881, 19)

In [21]:
mi400_col_names = ['rsrv_col_cd','clm_id','clm_num','clm_st_cd','st_asgn_st_cd','los_occr_dt','los_type_cd','rpt_agt_st_cd',
                   'los_loc_city_nm','los_loc_stret_nm','los_loc_st_cd','los_loc_desc_txt','los_desc_txt','los_rpt_dt',
                   'app_cd','user_type_cd','loc_qlty_cd','latud_num','lngtd_num']
mi400.columns = mi400_col_names
mi400.head()

,rsrv_col_cd,clm_id,clm_num,clm_st_cd,st_asgn_st_cd,los_occr_dt,los_type_cd,rpt_agt_st_cd,los_loc_city_nm,los_loc_stret_nm,los_loc_st_cd,los_loc_desc_txt,los_desc_txt,los_rpt_dt,app_cd,user_type_cd,loc_qlty_cd,latud_num,lngtd_num
0,400,0000052048071,004W253,22,,2010-12-28,REG,22,BASTTLE CREEK,,MI,PARKING RAMP,INSURED NOTICED DAMAGE WHEN RETURNING TO CAR,2011-01-06,I,AGT,,0,0
1,400,0000033413931,002V719,22,,2010-11-20,REG,,Bay City,,MI,probably in parking lot off EUCLID,"Final\rWhile V1 was parked and unoccupied, it ...",2010-11-20,E,CRC,,0,0
2,400,0000033413931,002V719,22,,2010-11-20,REG,,Bay City,,MI,probably in parking lot off EUCLID,"While V1 was parked and unoccupied, it was hit...",2010-11-20,E,CRC,,0,0
3,400,0000034590923,002Z262,22,,2010-11-23,REG,22,BENTON HARBOR,,MI,I-94 WEST BOUND,"V1 ON I94 NEAR EXIT RAMP. V2 STARTED TO EXIT,...",2010-11-24,I,AGT,,0,0
4,400,0000034590923,002Z262,22,,2010-11-23,REG,22,BENTON HARBOR,,MI,I-94 WEST BOUND,"V1 ON I94 NEAR EXIT RAMP. V2 STARTED TO EXIT,...",2010-11-24,I,AGT,,0,0


In [22]:
len(mi400.los_loc_city_nm.unique())

16766

## begin to assess the data
* how many unique claims?

In [24]:
len(mi400), len(mi400.clm_num.unique()),len(mi400.clm_id.unique())

(616881, 278762, 278763)

### why this discrepancy?


In [26]:
print mi400.rsrv_col_cd.unique()
# drop this
mi400.drop('rsrv_col_cd',axis=1,inplace=True)

[400]


In [27]:
mi400.st_asgn_st_cd.unique() # only 22 or blank

array(['  ', '22'], dtype=object)

In [28]:
mi400.clm_st_cd.unique()

array(['22', '13', '14', '01', '04', '36', '16', '47', '75', '24', '02',
       '18', '49', '55', '17', '43', '07', '03', '33', '25', '06', '50',
       '31', '42', '  ', '35', '12', '53', '37', '51', '23', '15', '27',
       '40', '59', '05', '38', '20', '34', '46', '60', '44', '09', '26',
       '28', '21', '11', '52', '64', '08', '19', '61', '30', '48', '45',
       '41', '32', '29', '39'], dtype=object)

In [29]:
mi400.los_loc_st_cd.unique()

array(['MI'], dtype=object)

In [30]:
mi400.los_type_cd.unique()

array(['REG', 'CNV', 'CAT', 'ADR', 'GLS', 'FTR', 'TOW'], dtype=object)

In [32]:
mi400.clm_num.tail()

616876    V287361
616877    V608876
616878    V627782
616879    V732324
616880    Z371410
Name: clm_num, dtype: object

In [33]:
mi400.clm_num.value_counts()

411Q889    14
496S732    12
422G620    12
5V01345    12
3R52252    12
5C96319    11
2P96481    11
453H416    11
413Q158    11
426K838    11
339G845    11
405M097    11
495G999    11
3N94034    11
5N29397    10
3T69512    10
516T220    10
5B27725    10
4V94740    10
4C40574    10
5J99193    10
3Q77588    10
403F677    10
361G051    10
3R24538    10
484L605    10
5R40250    10
4D23387    10
022V867    10
600N804    10
           ..
B092395     1
21M5412     1
150Q640     1
B092393     1
2525775     1
076N828     1
B139184     1
V700876     1
C418205     1
B186307     1
C402298     1
B183570     1
C418201     1
C418202     1
C450985     1
B183578     1
B130047     1
B193748     1
C409544     1
P278137     1
C400522     1
C409541     1
B084449     1
C446983     1
C446982     1
C409540     1
V376753     1
G663498     1
B191812     1
B155447     1
dtype: int64

In [38]:
mi400.columns

Index([u'clm_id', u'clm_num', u'clm_st_cd', u'st_asgn_st_cd', u'los_occr_dt',
       u'los_type_cd', u'rpt_agt_st_cd', u'los_loc_city_nm',
       u'los_loc_stret_nm', u'los_loc_st_cd', u'los_loc_desc_txt',
       u'los_desc_txt', u'los_rpt_dt', u'app_cd', u'user_type_cd',
       u'loc_qlty_cd', u'latud_num', u'lngtd_num'],
      dtype='object')

In [48]:
mi400[mi400.clm_num == '600N804'][['los_occr_dt','los_rpt_dt','los_type_cd','los_loc_city_nm','los_loc_desc_txt','loc_qlty_cd','latud_num']]

,los_occr_dt,los_rpt_dt,los_type_cd,los_loc_city_nm,los_loc_desc_txt,loc_qlty_cd,latud_num
445941,2015-02-08,2015-02-09,REG,BattleCreek,Beckley St.,GM,42.321152
445942,2015-02-08,2015-02-09,REG,BattleCreek,Becklie St.,,0.000000
445943,2015-02-08,2015-02-09,REG,BattleCreek,Becklie St.,GM,42.321152
445944,2015-02-08,2015-02-09,REG,BattleCreek,Shopping plaza with Target and Old Country Buf...,,0.000000
445945,2015-02-08,2015-02-09,REG,BattleCreek,Shopping plaza with Target and Old Country Buf...,GM,42.321152
445946,2015-02-08,2015-02-09,REG,BattleCreek,just outside Shopping plaza with Target and Ol...,,0.000000
445947,2015-02-08,2015-02-09,REG,BattleCreek,just outside Shopping plaza with Target and Ol...,,0.000000
445948,2015-02-08,2015-02-09,REG,BattleCreek,just outside Shopping plaza with Target and Ol...,AX3,42.261060
445949,2015-02-08,2015-02-09,REG,BattleCreek,just outside Shopping plaza with Target and Ol...,,0.000000
445950,2015-02-08,2015-02-09,REG,BattleCreek,just outside Shopping plaza with Target and Ol...,AX3,42.261060


In [44]:
mi400[mi400.clm_num=='4V94740'].los_loc_desc_txt.values

array(['EB McIntyre Road & SB Barrie Street',
       'EB McIntyre Road & SB Barrie Street',
       'EB McIntyre Road & SB Barrie Street',
       'EB McIntyre Road & SB Barrie Street',
       'Eastbound on MacIntyre Road & Barrie Street',
       'Eastbound on Macintire Road', 'Eastbound on Macintire Road',
       'Eastbound on Macintire Road & corss street is Barrie St',
       'EB McIntyre Road & SB Barrie Street',
       'EB McIntyre Road & SB Barrie Street'], dtype=object)

In [45]:
mi400[mi400.clm_num=='4V94740'].los_desc_txt.values

array([ 'IV e/b McIntyre @ IX Barrie w/o any traffic controls. CV s/b Barrie w/stop sign. IV enters IX',
       'IV e/b McIntyre @ IX Barrie w/o any traffic controls. CV s/b Barrie w/stop sign. IV enters IX and strikes CV.',
       'Matthew stated that he was driving EB on Macintire Rd approaching Barrie St when the claimant vehicle, traveling on Barrie St,  ran stop sign. The described vehicle struck the claimant vehicle on the passenger side. Both vehicles went into the ditch.  There were two fatalities from the claimant vehicle and one injured person that was life flighted to the hospital.',
       'NI stated that he was driving EB on Macintire Rd approaching Barrie St when V2 ran stop sign with V1 striking V2 on the passenger side with both vehicles ended up in the ditch. NI stated that he did not see V2 coming and did not have time to stop. NI stated that 2 people died in the accident and 1 more was life flighted to the hospital.',
       'CV ran stop sign and was hit by IV.',
   

## I need to work on the following item
1. aggregate notes for a given claim (different updates)
2. deal with std abbreviations:
    * NI
    * IV
    * IX
    * EB
    * CV
    * V1, V2, ...